In [1]:
import keras
from keras import layers
import numpy as np

Using TensorFlow backend.


In [2]:
latent_dim = 32
height = 32
width = 32
channels = 3

In [3]:
generator_input = keras.Input(shape=(latent_dim, ))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [4]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [5]:
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [6]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim, ))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

In [7]:
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

# Train our DCGAN

In [8]:
import os
from keras.preprocessing import image

(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

In [9]:
x_train = x_train[y_train.flatten() == 8] # ship images
x_train = x_train.reshape(
    (x_train.shape[0], ) + 
    (height, width, channels)).astype('float32') / 255. # normalizes data

In [10]:
iterations = 10000
batch_size = 20
save_dir = "E:\\temp\\dcgan"

In [11]:
start = 0

for step in range(iterations):
    # samples random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # generate fake images
    generated_images = generator.predict(random_latent_vectors)
    
    # get real images
    stop = start + batch_size
    real_images = x_train[start: stop]

    # create training data for discriminator
    combined_images = np.concatenate([generated_images, real_images]) 
    # label = 1 => fake, label = 0 => real world
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    
    # train discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    # samples random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # create fake targets (label 0 but it's from gan, not real world)
    misleading_targets = np.zeros((batch_size, 1))
    
    # train generator
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        
        print("discriminator loss: ", d_loss)
        print("adversarial loss: ", a_loss)
        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_ship' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_ship' + str(step) + '.png'))
        

E:\Miniconda3\envs\env3-gpu\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss:  0.667364
adversarial loss:  0.65996
discriminator loss:  0.310827
adversarial loss:  2.94974
discriminator loss:  0.783311
adversarial loss:  0.780401
discriminator loss:  0.694457
adversarial loss:  0.684183
discriminator loss:  0.698418
adversarial loss:  0.696844
discriminator loss:  0.692233
adversarial loss:  0.729003
discriminator loss:  0.704063
adversarial loss:  0.681386
discriminator loss:  0.70218
adversarial loss:  0.728991
discriminator loss:  0.700803
adversarial loss:  0.703575
discriminator loss:  0.708845
adversarial loss:  0.698349
discriminator loss:  0.694344
adversarial loss:  0.757346
discriminator loss:  0.692718
adversarial loss:  0.707597
discriminator loss:  0.696908
adversarial loss:  0.693149
discriminator loss:  0.6818
adversarial loss:  0.725513
discriminator loss:  0.696163
adversarial loss:  0.764356
discriminator loss:  0.695506
adversarial loss:  0.687942
discriminator loss:  0.691523
adversarial loss:  0.723755
discriminator loss: